<a href="https://colab.research.google.com/github/espada105/NewRecipe/blob/main/Llama3_2_11B_Vision_Instruct_NewRecipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>